In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
LEARNING_RATE = 1e-3
EPOCHS = 20
BATCH_SIZE = 8

In [3]:
imagePaths = list(paths.list_images('dataset'))
data = list()
labels = list()

for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    
    data.append(image)
    labels.append(label)
    
print('data: ',len(data), ' labels: ',len(labels))

data:  50  labels:  50


In [4]:
data = np.array(data) / 255.0
labels = np.array(labels)

# one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [5]:
trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.20, 
                                                stratify=labels, random_state=42)

trainAug = ImageDataGenerator(rotation_range=15, fill_mode="nearest")

In [6]:
baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
for layer in baseModel.layers:
    layer.trainable = False

In [8]:
print("compiling model")
opt = Adam(lr=LEARNING_RATE, decay=LEARNING_RATE / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

compiling model


In [ ]:
print("train head")
H = model.fit_generator(
    trainAug.flow(trainX, trainY, batch_size=BATCH_SIZE),
    steps_per_epoch=len(trainX) // BATCH_SIZE,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BATCH_SIZE,
    epochs=EPOCHS
)

train head
Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
5/5 [==============================] - 8s 2s/step - loss: 0.6569 - acc: 0.6250 - val_loss: 0.7648 - val_acc: 0.5000
Epoch 2/20
5/5 [==============================] - 7s 1s/step - loss: 0.7612 - acc: 0.4000 - val_loss: 0.7413 - val_acc: 0.6000
Epoch 3/20
5/5 [==============================] - 8s 2s/step - loss: 0.7034 - acc: 0.5500 - val_loss: 0.7306 - val_acc: 0.7000
Epoch 4/20
5/5 [==============================] - 8s 2s/step - loss: 0.6656 - acc: 0.6000 - val_loss: 0.7138 - val_acc: 0.8000
Epoch 5/20
5/5 [==============================] - 8s 2s/step - loss: 0.6841 - acc: 0.5500 - val_loss: 0.7003 - val_acc: 0.7000
Epoch 6/20
5/5 [==============================] - 8s 2s/step - loss: 0.6432 - acc: 0.6750 - val_loss: 0.6925 - val_acc: 0.8000
Epoch 7/20
5/5 [==============================] - 8s 2s/step - loss: 0.6005 - acc: 0.7000 - val_loss: 0.6899 - val_acc: 0.8000
Epoch 

In [ ]:
print("evaluation")
predIdxs = model.predict(testX, batch_size=BATCH_SIZE)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testY.argmax(axis=1), predIdxs, target_names=lb.classes_))

In [ ]:
N = EPOCHS
plt.style.use("ggplot")
plt.figure(figsize=(12, 8))
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")

print('model saved')
model.save("covid19.model", save_format="h5")